In [1]:
import autograd.numpy as np
from autograd import grad, jacobian, hessian
from pyDOE import lhs
#from basis_11 import xdes
import matplotlib.pyplot as plt
from random import choices
import numpy as npp



a=0.001 # alpha
b_1=0.9 # beta1
b_2=0.999 # beta2
N=3000;
n1=50;
n2=50;
e=10**-7;

Ntt=500;#testing points

def generate_x(N):
    x1 = 4*lhs(1,N)+50;
    x2 = 4*lhs(1,N)+50;
    y = np.multiply(np.cos(np.pi*x1),np.cos(np.pi*x2))
    return x1, x2, y

def generate_rand():
    weight1 = np.random.normal(0,2/(2+n1),(n1,2));
    weight2 = np.random.normal(0,2/(n1+n2),(n2,n1));
    weight3 = np.random.normal(0,2/(1+n2),(1,n2));
    bias1 = np.random.random_sample([50,1]);
    bias2 = np.random.random_sample([50,1]);
    return weight1, weight2, weight3, bias1, bias2

def norm_weights(x,y):
    mean = np.mean(x,axis=0);
    variance = np.var(x,axis=0);
    meany = np.mean(y);
    variancey = np.var(y);
    x1=np.array((x[:,0]-mean[0])/np.sqrt(variance[0]+e))[:,None];
    x2=np.array((x[:,1]-mean[1])/np.sqrt(variance[1]+e))[:,None];
    y=np.array((y-meany)/np.sqrt(variancey+e))
    return x1, x2, y

def l2_norm(x,weight1,weight2,weight3,bias1,bias2):
    a1=np.matmul(weight1,x)+bias1;
    y1=np.tanh(a1);
    a2=np.matmul(weight2,y1)+bias2;
    y2=np.tanh(a2);
    y3=np.matmul(weight3,y2);
    y4=np.sum((y.T-y3)**2)+0*(np.sum(weight3**2)+np.sum((weight2)**2)+np.sum(weight1**2))
    
    return y4

def rel_l2_norm(x,weight1,weight2,weight3,bias1,bias2):
    a1=np.matmul(weight1,x)+bias1;
    y1=np.tanh(a1);
    a2=np.matmul(weight2,y1)+bias2;
    y2=np.tanh(a2);
    y3=np.matmul(weight3,y2);
    y4=np.sum((y.T-y3)**2)/np.sum((y)**2)
    
    return y4


def rel_l2_norm_test(x,weight1,weight2,weight3,bias1,bias2):
    a1=np.matmul(weight1,x)+bias1;
    y1=np.tanh(a1);
    a2=np.matmul(weight2,y1)+bias2;
    y2=np.tanh(a2);
    y3=np.matmul(weight3,y2);
    y4=np.sum((yT.T-y3)**2)/np.sum((yT)**2)
    
    return y4

def activation(a2):
    y=np.tanh(a2);
    return y

qy=[];
x1, x2, y=generate_x(N);

weight1 ,weight2, weight3, bias1, bias2 =generate_rand();
x=np.hstack((x1,x2));
#x=x.T;
x1,x2,y=norm_weights(x,y);
x1=x1.T;
x2=x2.T;
x=np.vstack((x1,x2));
jacobian_w3=jacobian(l2_norm,3);
jacobian_w2=jacobian(l2_norm,2);
jacobian_b2=jacobian(l2_norm,5);
jacobian_w1=jacobian(l2_norm,1);
jacobian_b1=jacobian(l2_norm,4);

vw1=0;
mw3=0;
vw3=0;
mw2=0;
vw2=0;
mw1=0;
mb1=0;
vb1=0;
mb2=0;
vb2=0;


#

for t in range(1,5000):
#t=1;


    wu3=np.reshape(jacobian_w3(x,weight1,weight2,weight3,bias1,bias2),(1,n2));
    wu2=np.reshape(jacobian_w2(x,weight1,weight2,weight3,bias1,bias2),(n2,n1));
    wu1=np.reshape(jacobian_w1(x,weight1,weight2,weight3,bias1,bias2),(n1,2));
    b1=np.reshape(jacobian_b1(x,weight1,weight2,weight3,bias1,bias2),(n1,1));
    b2=np.reshape(jacobian_b2(x,weight1,weight2,weight3,bias1,bias2),(n1,1));
    
    mw3 = b_1*mw3 + (1-b_1)*wu3;
    vw3 = b_2*vw3 + (1-b_2)*np.power(wu3,2);
      
    mw3t=mw3/(1-np.power(b_1,t));
    vw3t=vw3/(1-np.power(b_2,t));
    
    mw2 = b_1*mw2 + (1-b_1)*wu2;
    vw2 = b_2*vw2 + (1-b_2)*np.power(wu2,2);
      
    mw2t=mw2/(1-np.power(b_1,t));
    vw2t=vw2/(1-np.power(b_2,t));
    
    mw1 = b_1*mw1 + (1-b_1)*wu1;
    vw1 = b_2*vw1 + (1-b_2)*np.power(wu1,2);
      
    mw1t=mw1/(1-np.power(b_1,t));
    vw1t=vw1/(1-np.power(b_2,t));
    
    mb1 = b_1*mb1 + (1-b_1)*b1;
    vb1 = b_2*vb1 + (1-b_2)*np.power(b1,2);
      
    mb1t=mb1/(1-np.power(b_1,t));
    vb1t=vb1/(1-np.power(b_2,t));
    
    mb2 = b_1*mb2 + (1-b_1)*b2;
    vb2 = b_2*vb2 + (1-b_2)*np.power(b2,2);
      
    mb2t=mb2/(1-np.power(b_1,t));
    vb2t=vb2/(1-np.power(b_2,t));
       
    weight3 = weight3 - a*np.divide(mw3t,np.sqrt(vw3t)+e);
    weight2 = weight2 - a*np.divide(mw2t,np.sqrt(vw2t)+e);
    weight1 = weight1 - a*np.divide(mw1t,np.sqrt(vw1t)+e);
    bias1=bias1 - a*np.divide(mb1t,np.sqrt(vb1t)+e);
    bias2=bias2 - a*np.divide(mb2t,np.sqrt(vb2t)+e);
    
    qy=np.append(qy,rel_l2_norm(x,weight1,weight2,weight3,bias1,bias2));
        
plt.plot(qy)


#testing

x1T,x2T,yT=generate_x(Ntt)
xT=np.hstack((x1T,x2T));
x1T,x2T,yT=norm_weights(xT,yT);
x1T=x1T.T;
x2T=x2T.T;
xT=np.vstack((x1T,x2T));
fin_1=rel_l2_norm_test(xT,weight1,weight2,weight3,bias1,bias2)
print(fin_1)

0.0005209995969336232
